# Introduction

* Datasets:
    * https://huggingface.co/datasets/timdettmers/openassistant-guanaco
* Models:
    * https://huggingface.co/facebook/opt-125m

In [1]:
!pip install -U accelerate transformers trl datasets

In [2]:
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

## Configuration

In [3]:
batch_size = 2
num_workers = os.cpu_count()
# max_steps = -1 for epoch-wise training.
# epochs = -1 for step-wise training.
# Both cannot be -1.
max_steps = -1
epochs = 3
bf16 = True
fp16 = False
gradient_accumulation_steps = 16
seq_length = 1024
logging_steps = 50
save_steps = 50
learning_rate = 0.0002
model_name = 'facebook/opt-125m'
out_dir = 'outputs/opt_125m_openassistant_guanaco_sft'
seed = 42

## Load Dataset 

In [4]:
dataset = load_dataset('timdettmers/openassistant-guanaco')

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})


In [6]:
print(dataset['train']['text'][0])

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

## Model

In [7]:
if bf16:
    model = AutoModelForCausalLM.from_pretrained(model_name).to(dtype=torch.bfloat16)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

## Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True,
    use_fast=False
)

## Training

In [10]:
if max_steps == -1 and epochs > 0:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='epoch',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='epoch',
        logging_steps=logging_steps,
        num_train_epochs=epochs,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

if max_steps > 0 and epochs == -1:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='steps',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='steps',
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        max_steps=max_steps,
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
        seed=seed
    )

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field='text',
    max_seq_length=seq_length,
    tokenizer=tokenizer,
    args=training_args,
    packing=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
dataloader = trainer.get_train_dataloader()
for i, sample in enumerate(dataloader):
    print(tokenizer.decode(sample['input_ids'][0]))
    print('#'*50)
    if i == 5:
        break

�代理规则。在代理规则中，输入要代理的服务的端口和主机地址。

配置DNS解析：在代理规则配置完成后，需要配置DNS解析。如果要代理的服务不在本地网络中，需要将其域名解析到Nginx Proxy Manager所在的服务器IP地址上。可以在DNS服务器中配置相应的解析记录，或者在本地Hosts文件中添加相应的解析记录。

测试代理服务：完成上述步骤后，可以通过浏览器访问你所代理的服务。在浏览器中输入代理服务的域名或IP地址，应该能够正常访问服务。</s></s>### Human: ¿Puedes enumerar las 5 características más importantes que tienen los insectos?### Assistant: - Los insectos son un tipo de animales invertebrados que pertenecen al filo de los artrópodos. Se denominan así por poseer patas articuladas, al igual que los crustáceos, los arácnidos o los extintos trilobites. 
- Normalmente los insectos tienen 3 pares de patas pero, en casos como el del milpies, pueden tener hasta 400 pares.
- El cuerpo de los insectos está dividido en tres secciones: cabeza, tórax y abdomen. 
- Todo su cuerpo está cubierto por un exoesqueleto formado por quitina, una sustancia dura y flexible.
- Los insectos en su etapa adulta tienen par de antenas sensoriales situadas en la cabeza.</s></s>### Human: ¿Cual es el proceso de fabricació

In [13]:
history = trainer.train()

Epoch,Training Loss,Validation Loss
0,2.139900,2.056763
1,1.923600,2.003074
2,1.804000,1.981635


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [14]:
model.save_pretrained(f"{out_dir}/best_model")
tokenizer.save_pretrained(f"{out_dir}/best_model")

('outputs/opt_125m_openassistant_guanaco_sft/best_model/tokenizer_config.json',
 'outputs/opt_125m_openassistant_guanaco_sft/best_model/special_tokens_map.json',
 'outputs/opt_125m_openassistant_guanaco_sft/best_model/vocab.json',
 'outputs/opt_125m_openassistant_guanaco_sft/best_model/merges.txt',
 'outputs/opt_125m_openassistant_guanaco_sft/best_model/added_tokens.json')

## Inference

In [1]:
from transformers import (
    AutoModelForCausalLM, 
    logging, 
    pipeline,
    AutoTokenizer
)

import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
model = AutoModelForCausalLM.from_pretrained('outputs/opt_125m_openassistant_guanaco_sft/best_model/')
tokenizer = AutoTokenizer.from_pretrained('outputs/opt_125m_openassistant_guanaco_sft/best_model/')

In [4]:
pipe = pipeline(
    task='text-generation', 
    model=model, 
    tokenizer=tokenizer, 
    max_length=300, # Prompt + new tokens to generate.
    device_map=device,
    eos_token_id=tokenizer.eos_token_id
)

In [5]:
logging.set_verbosity(logging.CRITICAL)

In [6]:
template = '### Human: {}### Assistant:'

In [8]:
prompt = "How to stay healthy?"
prompt = template.format(prompt)

outputs = pipe(
    prompt, 
    # do_sample=True, 
    # temperature=0.7, 
    # top_k=50, 
    # top_p=0.95,
    repetition_penalty=1.1,
)
print(outputs[0]['generated_text'])

### Human: How to stay healthy?### Assistant: There are several steps you can take to stay healthy:

1. Start exercising regularly: Regular exercise is essential for maintaining a healthy body and mind. Aim to walk, run, or bike for at least 30 minutes each day.
2. Get enough sleep: Sleep is important for physical and mental health. Aim to get enough sleep every night.
3. Stay hydrated: Drinking plenty of water throughout the day can help regulate your body's natural energy levels.
4. Eat healthy foods: Eating healthy foods like vegetables, fruits, and whole grains can help reduce stress and improve overall health.
5. Exercise regularly: Exercise is essential for physical and mental health. Aim to exercise regularly to improve your physical and mental health.
6. Stay hydrated: Drinking plenty of water throughout the day can help regulate your body's natural energy levels.
7. Stay hydrated: Drinking plenty of water throughout the day can help regulate your body's natural energy levels.
